In [217]:
import numpy as np

class KalmanFilter:
    def __init__(self, 
    #-- State Model--#
    F, # State Transition Matrix
    G, # Control Input Matrix
    Q, # Process Noise Covariance
    #-- Measurement Model--#
    H, # Measurement Matrix
    R, # Measurement Noise Covariance
    #-- Initial State--#
    P, # Initial Estimate Covariance
    x  # Initial State Estimate
    ):
        # initialize the state model
        self.F = F
        self.G = G
        self.Q = Q
        self.H = H
        self.R = R
        self.P = P
        self.x = x

        # initialize the state estimate
        self.x_hat = self.x
        self.P_hat = self.P

        self.x_hat_prev = self.x_hat
        self.P_hat_prev = self.P_hat

        # Convert numpy arrays to matrices
        self.F = np.matrix(self.F)
        self.G = np.matrix(self.G)
        self.Q = np.matrix(self.Q)
        self.H = np.matrix(self.H)
        self.R = np.matrix(self.R)
        self.P = np.matrix(self.P)
        self.x = np.matrix(self.x)
        self.x_hat = np.matrix(self.x_hat)
        self.P_hat = np.matrix(self.P_hat)
        self.x_hat_prev = np.matrix(self.x_hat_prev)
        self.P_hat_prev = np.matrix(self.P_hat_prev)
    
    def update(self, 
        z, # Measurement
    ):
        # Predict state estimate
        # F*Xt-1 + G*Ut
        self.x_hat = self.F * self.x_hat_prev # No control input

        # Predict estimate covariance
        # F*Pt-1*F^T + Q
        self.P_hat = self.F * self.P_hat_prev * self.F.T + self.Q

        # Calculate Kalman Gain
        # P*H^T*(H*P*H^T + R)^-1
        K = self.P_hat * self.H.T * np.linalg.inv(self.H * self.P_hat * self.H.T + self.R)

        # Update state estimate
        # Xt = Xt + K*(Zt - H*Xt)
        self.x_hat = self.x_hat + K * (z - self.H * self.x_hat)

        # Update estimate covariance
        # (I - K*H)*P
        self.P_hat = (np.eye(self.P.shape[0]) - K * self.H) * self.P_hat

        # Update previous state estimate and covariance
        self.x_hat_prev = self.x_hat
        self.P_hat_prev = self.P_hat

        return self.x_hat

    
    def get_state_estimate(self):
        return self.x_hat


# X,y,Vx,Vy,dt state model  
dt = 1/30

F = np.array([[1, 0, dt, 0],
                [0, 1, 0, dt],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])

G = np.array([[0.5*dt**2, 0],
                [0, 0.5*dt**2],
                [dt, 0],
                [0, dt]])

Q = np.array([[0.1, 0, 0, 0],
                [0, 0.1, 0, 0],
                [0, 0, 0.1, 0],
                [0, 0, 0, 0.1]])

# X,y state model
H = np.array([[1, 0, 0, 0],
                [0, 1, 0, 0]])

R = np.array([[0.01, 0],
                [0, 0.01]])

P = np.array([[0.1, 0, 0, 0],
                [0, 0.1, 0, 0],
                [0, 0, 0.1, 0],
                [0, 0, 0, 0.1]])

x = np.array([[0],
                [0],
                [0],
                [0]])

kf = KalmanFilter(F, G, Q, H, R, P, x)

In [218]:
# Example observation x,y
z = np.array([  [1], # x
                [1]]# y
                )



In [219]:
z += 1

In [244]:
print(f'z: {z}')

kf.update(z)


z: [[2]
 [2]]


matrix([[2.00008793],
        [2.00008793],
        [0.02879145],
        [0.02879145]])

In [357]:
# Update the state estimate using control input
u = np.array([[1.0], [1.0]])
kf.update(z, u)

matrix([[2.00292322],
        [2.00292322],
        [0.97692647],
        [0.97692647]])

In [221]:
state_estimate = kf.get_state_estimate()

print(f'state_estimate: {state_estimate}')

state_estimate: [[1.90481227]
 [1.90481227]
 [0.03172924]
 [0.03172924]]


In [222]:
# # Example control input Vx,Vy
# u = np.array([[1],
#                 [1]])

# kf.predict()